In [1]:
import json

In [2]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)


documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [3]:
documents[0]

{'text': "dThe purpose of this document is to capture frequently asked technical questions\nThe next cohort starts in Jan 2025. More info at DTC Article.\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start？',
 'course': 'data-engineering-zoomcamp'}

In [4]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

/home/codespace/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/codespace/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
indexd_text =[]

for doc in documents:
    doc['text_vector'] = model.encode(doc['text']).tolist()
    indexd_text.append(doc)

In [11]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200')

es_client.info()

ObjectApiResponse({'name': '934a4bc7a925', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'UGFBXMLRQrqh9_e0sByKvA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [15]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0,
},
    "mappings": {
        "properties": {
            "text" : {"type": "text"},
            "section" : {"type": "text"},
            "question" : {"type" : "text"},
            "course": {"type" : "keyword"},
            "text_vector": {"type" :"dense_vector", "dims": 386, "index": True, "similarity": "cosine"},
            
        }
    }
}

In [16]:
index_name = "course-questions"

#as a best practice after updating or indexing or reindex first delete (if there is with the same name) to avoid error
es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body= index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})